### 1- IMPORT LIBRARIES & DATA

In [ ]:
# !pip install dash
# !pip install pandas 
# !pip install dash-bootstrap-components
# !pip install pymysql
# !pip install Flask-Caching
# !pip install mysqlclient
# !pip install dash-extensions
# !pip install mysql-connector-python

In [1]:
import os
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import plotly.graph_objs as go
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dash_table
from dash.dash_table.Format import Group
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State, ClientsideFunction, MATCH, ALL, ClientsideFunction, Output, Input
from dash.exceptions import PreventUpdate
# from dash_extensions import Download
# from dash_extensions.snippets import send_data_frame
import dash_bootstrap_components as dbc
from navbar_tabs_layout import app_layout
import time
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import relativedelta
import mysql.connector
import pymysql
pymysql.install_as_MySQLdb()
#import MySQLdb
from flask_caching import Cache
from dash_extensions.enrich import DashProxy, Output, Input, State, ServersideOutput, html, dcc, ServersideOutputTransform
#from app import app, cache

/Users/laetitiahoquetis/Downloads/Python/Dashboard/navbar_tabs_layout.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/Users/laetitiahoquetis/Downloads/Python/Dashboard/navbar_tabs_layout.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [3]:
# Read the data
df = pd.read_csv('EZItotalFull.csv')

In [4]:
# Call data preprocessing function to preprocess data
data = None  # Set global_data to None to force a reload of data

### 2- LAUNCH DASH 

In [5]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.UNITED], meta_tags=[{"name": "viewport", "content": "width=device-width"}])
app.title = 'Dashboard'

In [6]:
# Create a Flask-Caching object
cache = Cache(app.server, config={
    'CACHE_TYPE': 'filesystem',
    'CACHE_DIR': 'cache-directory',
    'CACHE_DEFAULT_TIMEOUT': 86400  # 24 hours
})
cache.clear()

True

In [7]:
# Create a dcc.Store component to store the data
store = dcc.Store(id='local', storage_type='local')

### 4- PREPROCESS CLEAN DATA

In [8]:
# define a function to preprocess the data
def preprocess_data(df):
    df["date"] = df["date"].astype('datetime64[ns]')
    #df["date"] = pd.to_datetime(df["date"])
    df[['year', 'month', 'day', 'weekday']] = df['date'].apply(lambda x: pd.Series([x.year, x.month, x.day, x.day_name()]))
    df[['quantity', 'product_id', 'customer_company_id', 'id']] = df[['quantity', 'product_id', 'customer_company_id', 'id']].astype('Int64')
    df['face_price'] = df['face_price'].astype('float').round(2)
    df['price'] = df['external_unit_price'].str.extract('(\d+)').astype('float') / 100
    df['price'] = df['price'].round(2)
    df['total'] = (df['quantity'] * df['price']).round(2)
    df.sort_values(['date','id'],ascending=[True, True])
    return df


### 3- LOAD DATA or REFRESH

In [9]:
@cache.memoize()
def read_data():
    # Try to get data from the cache
    data = cache.get('data')
    if data is not None:
        return pd.DataFrame.from_dict(data)

    # Try to load data from the CSV file
    if os.path.isfile('EZItotalFull.csv'):
        data = pd.read_csv('EZItotalFull.csv')
        data = preprocess_data(data)
        # Store the data in the cache
        cache.set('data', data.to_dict('records'))
        #return pd.DataFrame.from_dict(data)
        return data

In [10]:
#read_data().tail()

In [11]:
# Define connection parameters

# Set up the database connection
def get_database_connection():
    connection = pymysql.connect(
        host = '127.0.0.1',
        user = 'readonly',
        password = 'MwLvD9DFL8mnrTKkI6fU',
        port = 3335
        #DB_NAME = 'mydatabase',
    )
    return connection

In [12]:
# Read data from the database and store it in cache
def fetch_data_from_database():
    connection = get_database_connection()
    cursor = connection.cursor()
    # Get the last sales order ID from the CSV file
    if os.path.isfile('EZItotalFull.csv'):
        df = pd.read_csv('EZItotalFull.csv')
        last_date = df['created_at'].astype('datetime64[ns]').max().strftime('%Y-%m-%d')
        last_date_o = df['date'].astype('datetime64[ns]').max().strftime('%Y-%m-%d')
        last_sales_order_id = pd.to_numeric(df['sales_order_id'], errors='coerce').max()
        last_inv = pd.to_numeric(df['invoice_number'], errors='coerce').max()
    else:
        df = pd.DataFrame()  # create an empty dataframe if the CSV file doesn't exist
        last_date = "2019-01-01"
        last_date_o = "2019-01-01"
    query = f"""
                SELECT i.sales_order_id, i.id AS id, i.quantity, i.product_id, i.state AS state_i, i.status, i.created_at, i.external_unit_price,
                       p.id AS sku_id, p.sku, p.name, p.description, p.currency, p.brand_id, p.face_price, p.country, b.name AS name_i,
                       o.date, o.invoice_number, o.state AS state_o, o.customer_company_id, c.id AS cust_id, c.name AS name_o, a.city, a.country AS country_o, x.finance_number, x.sales_order_id AS sales_order_id_o
                        FROM (SELECT *
                              FROM ezscm_production.sales_order_items
                              WHERE created_at > '{last_date}')  AS i
                        LEFT OUTER JOIN (SELECT *
                                         FROM ezscm_production.sales_orders
                                         WHERE date > '{last_date_o}') AS o
                            ON o.id = i.sales_order_id
                        LEFT OUTER JOIN ezscm_production.products AS p
                            ON p.id = i.product_id
                        LEFT OUTER JOIN ezscm_production.brands AS b
                            ON p.brand_id = b.id
                        LEFT OUTER JOIN ezscm_production.companies AS c
                            ON o.customer_company_id = c.id
                        LEFT OUTER JOIN ezscm_production.addresses AS a
                            ON c.billing_address_id = a.id
                        LEFT OUTER JOIN ezscm_production.invoices AS x
                            ON o.invoice_number = x.finance_number
                """
    #connect cursor
    cursor.execute(query)
    rows = cursor.fetchall()
    data = pd.read_sql(query, connection)
    # Append new records to the CSV file
    df = pd.concat([df, data]).drop_duplicates().reset_index(drop=True)
    df = preprocess_data(df)
    df.to_csv('df.csv', index=False)
    cache.set('data', df.to_dict('records'))
    connection.close()
    
    #return pd.DataFrame.from_dict(df)
    return df


In [13]:
#fetch_data_from_database().tail()

In [14]:
@app.callback(Output('local', 'data'),
              Output("loading-fetch-data", "children"),
              Input('fetch-data-button', 'n_clicks'))

def fetch_data(n_clicks):
    if n_clicks is None or n_clicks == 0:
    # Try to load data from the cache or CSV file
        data = read_data()
        #data = preprocess_data(data)
        #cache.set('data', data.to_dict('records'))
        if data.empty:
            return pd.DataFrame(), html.Div([
                html.P('Data not found. Click on "Fetch Data" button.')
            ])
        else:
            return data.to_dict('records'), ''
    else:
        # Fetch data from the database and convert to pandas DataFrame
        try:
            #data = fetch_data_from_database()
            data = pd.read_csv('df.csv')
            # preprocess the data
            #data = preprocess_data(data)

            #store data in cache
            #cache.set('data', data.to_dict('records'))

            return data.to_dict('records'), html.Div([
                html.P('Data fetched successfully')
            ])
        except Exception as e:
            return pd.DataFrame(), html.Div([
                html.H6('Connect to MySQL please', style={'color': 'red'})
            ])


In [15]:
#fetch_data(1)[0][-1:]

In [16]:
# df["date"] = df["date"].astype('datetime64[ns]')
# df['year'] = df.date.dt.year.astype('Int64')
# df['month'] = df.date.dt.month.astype('Int64')
# df['day'] = df.date.dt.day.astype('Int64')
# df['weekday'] = df.date.dt.day_name()

In [17]:
# df = df.sort_values(['date','id'],ascending=[True, True])

In [18]:
# #cleaning other columns to be exploitable
# df['quantity'] = pd.to_numeric(df['quantity'], downcast="integer") 
# df['product_id'] = pd.to_numeric(df['product_id'], downcast="integer") 
# df['customer_company_id'] = pd.to_numeric(df['customer_company_id'], downcast="integer") 
# df['id'] = pd.to_numeric(df['id'], downcast="integer") 
# df['face_price'] = round(pd.to_numeric(df['face_price'], downcast="float"),2) 

In [19]:
# #creating a column that states clearly the unit price in a clean way : currency and price
# df['price'] = df['external_unit_price'].str.extractall('(\d+)').unstack().loc[:,0]
# df['price'] = pd.to_numeric(df['price'], downcast='float')
# df['price'] = df['price']/100
# df['price'] = round(df['price'], 2)

In [20]:
# #creating a order value column : quantity * price
# df['total'] = df['quantity']*df['price']
# df['total'] = round(df['total'],2)

### 5- DISPLAY RESPONSE 

In [21]:
# @app.callback(Output('table-container', 'children'),
#               [Input('fetch-data-button', 'n_clicks')])
# def update_data(n_clicks):
#     # Try to load data from the cache
#     data = cache.get('data')
#     if data is None:
#         return html.Div([
#             html.P('Please click on the "Fetch Data" button to fetch the data.')
#         ])
    
#     # preprocess the data
#     data = pd.DataFrame.from_records(data)
#     data = preprocess_data(data)

#     return dbc.Container([
#         html.H3('Last row of Data Table'),
#         dash_table.DataTable(
#             id='table',
#             columns=[{"name": i, "id": i} for i in data.columns],
#             data=[data.iloc[-1].to_dict()],
#             style_cell={'textAlign': 'center'},
#             style_header={
#                 'backgroundColor': 'rgb(230, 230, 230)',
#                 'fontWeight': 'bold'
#             },
#             style_data_conditional=[
#                 {
#                     'if': {'row_index': 'odd'},
#                     'backgroundColor': 'rgb(248, 248, 248)'
#                 }
#             ]
#         )
#     ])


In [22]:
@app.callback(Output('table-container', 'children'),
              [Input('fetch-data-button', 'n_clicks'),
               Input('local', 'data')])
def update_data(n_clicks, data):
    if n_clicks is None or n_clicks == 0:
        # Try to load data from the cache or CSV file
        data = read_data()
        if data.empty:
            return html.Div([
                html.P('Please click on the "Fetch Data" button to fetch the data.')
            ])
        else:
            #cache.set('data', data.to_dict('records'))
            pass 
    else:
        # Fetch data from the database and convert to pandas DataFrame
        #data = pd.DataFrame.from_dict(fetch_data_from_database())
        #fetch_data(1)[0]
        data = pd.read_csv('df.csv')
    
#     # preprocess the data
#     data = preprocess_data(data)

#     #store data in cache
#     cache.set('data', data.to_dict('records'))

    return dbc.Container([
        html.H3('Last row of Data Table'),
        dash_table.DataTable(
            id='table',
            columns=[{"name": i, "id": i} for i in data.columns],
            data=[data.iloc[-1].to_dict()],
            style_cell={'textAlign': 'center'},
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                }
            ]
        )
    ])
#     # convert data to a JSON serializable dictionary
#     data_dict = data.to_dict(orient='split')
#     columns = [{"name": i, "id": i} for i in data.columns]
#     table_data = data_dict['data'][-1]

#     return dbc.Container([
#         html.H6('Last row of Data Table'),
#         dash_table.DataTable(
#             id='table',
#             columns=columns,
#             data=table_data,
#             style_cell={'textAlign': 'center'},
#             style_header={
#                 'backgroundColor': 'rgb(230, 230, 230)',
#                 'fontWeight': 'bold'
#             },
#             style_data_conditional=[
#                 {
#                     'if': {'row_index': 'odd'},
#                     'backgroundColor': 'rgb(248, 248, 248)'
#                 }
#             ]
#         )
#     ])

In [23]:
#update_data(1, data)

### 6- NAVBAR LAYOUT 

In [24]:
today_date = datetime.now().strftime("%Y-%m-%d")

In [25]:
navbar = dbc.Navbar(
    dbc.Container(
        [
            dbc.Row(
                [
                    dbc.Col(html.Img(src="db.png", height="20px"), width="auto", align="center"),
                    dbc.Col(dbc.NavbarBrand("My Dashboard", className="ml-2"), width="auto", align="center"),
                    dbc.Col(html.Div(id="today-date", children=today_date), width="auto", align="center"),
                    dbc.Col(
                        [
                            dbc.Button('Fetch Data', id='fetch-data-button', n_clicks=0, className="mr-2"),
                     dcc.Loading(
                                id="loading-fetch-data",
                                type="default",
                                children=[
                                    dbc.Button(id='query-status-button', children='',style={'width': 'auto', 'margin-left': '10px'})
                                ]
                            ),
                            #dcc.Store(id='local', storage_type='local'),
                            html.Div(id='status'),
                        ],
                        width="auto", align="end",style={'display': 'flex', 'align-items': 'center'}
                    ),
                ],
                className="my-row",
                align="center",
            ),
            dbc.NavbarToggler(id="navbar-toggler"),
        ]
    ),
    color="light",
    dark=False,
    sticky="top",
)

In [26]:
# #what percentage of not complete orders do we have
# unsuccess = df[df['state_o']!='complete']
# unsuccess.groupby('name_o').agg({'quantity':'count','total':'sum'}).sort_values(by='quantity', ascending= False).head(5)

In [27]:
# df.tail()

### 8- PAGE BODY : TOP LEVEL VISUALS

In [28]:
# data_dict = cache.get('data')
# data_df = pd.DataFrame.from_dict(data_dict)
# data_df.tail()

In [29]:
# Define page layout
page_layout = dbc.Container([
    
    # Add top_level section
    dbc.Row([
                dbc.Col(dcc.DatePickerSingle(
                    id='date-picker',
                    # min_date_allowed=None,#data_df['date'].min(),
                    # max_date_allowed=None,#data_df['date'].max(),
                    # initial_visible_month=None,#data_df['date'].max(),
                    # date=None, #data_df['date'].max()
                ),
                width=3
        ),
        dbc.Col(dbc.Card([
                    dbc.CardHeader("Overnight Orders, including not Complete"),
                    dbc.CardBody([html.P(id='overnight-sales')])
                ]),
                width=8
        ),

    ], justify="between", align="center", className='mb-4'),

    dbc.Card([
        dbc.CardBody([
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardHeader("Sales YtD"),
                        dbc.CardBody([
                            dcc.Graph(id='sales-chart-2')
                        ])
                    ], className="h-100", style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'})
                ], sm=12, lg=4, className="mb-4"),
                dbc.Col([
                    dbc.Card([
                        dbc.CardHeader("Sales Mtd"),
                        dbc.CardBody([
                            dcc.Graph(id='sales-chart')
                        ])
                    ], className="h-100", style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'})
                ], sm=12, lg=4, className="mb-4"),
                dbc.Col([
                    dbc.Card([
                        dbc.CardHeader(
                            [html.H6("Customers to chase"), html.Button('Render Table', id='render-button', className='btn-sm')],
                            style={'backgroundColor': '#F8F9FA'}
                        ),
                        dbc.CardBody(
                            dcc.Graph(
                                id='table',
                                config={'displayModeBar': False}
                            ),
                            style={'padding': '0', 'margin': '0'}
                        )
                    ], className="h-100", style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'})
                ], sm=12, lg=4, className="mb-4")
            ])
        ])
    ], className="rounded-0 border-0"),

    # Add rest of page layout here
    html.Div("Click on below tabs for further analysis 👇"),

], fluid=True, style={'padding': '2rem'})


In [30]:
# # Define callback to read data into cache
# @app.callback(
#     dash.dependencies.Output('cache', 'data'),
#     dash.dependencies.Input('date-picker', 'date')
# )
# def data_callback(date):
#     # Load data and preprocess it
#     data_df = pd.read_csv('df.csv')
#     data_df = preprocess_data(data_df)

#     # Store data in cache
#     return data_df.to_dict('records')


In [35]:
@app.callback(
    Output('date-picker', 'min_date_allowed'),
    Output('date-picker', 'max_date_allowed'),
    Output('date-picker', 'initial_visible_month'),
    Output('date-picker', 'date'),
    Input('local', 'data')
)
def update_datepicker(data):
    if data is None:
        raise PreventUpdate
    
    data_df = pd.DataFrame.from_dict(data)
    min_date_allowed = data_df['date'].min().strftime('%Y-%m-%d')
    max_date_allowed = data_df['date'].max().strftime('%Y-%m-%d')
    initial_visible_month = min_date_allowed
    date = data_df['date'].max().strftime('%Y-%m-%d')

    return min_date_allowed, max_date_allowed, initial_visible_month, date


Custom TB Handler failed, unregistering


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
    [... skipping hidden 1 frame]

Cell In [37], line 1
----> 1 update_datepicker(read_data().tail)

Cell In [35], line 12, in update_datepicker(data)
     10     raise PreventUpdate
---> 12 data_df = pd.DataFrame.from_dict(data)
     13 min_date_allowed = data_df['date'].min().strftime('%Y-%m-%d')

File /usr/local/lib/python3.10/site-packages/pandas/core/frame.py:1764, in DataFrame.from_dict(cls, data, orient, dtype, columns)
   1763 if orient != "tight":
-> 1764     return cls(data, index=index, columns=columns, dtype=dtype)
   1765 else:

File /usr/local/lib/python3.10/site-packages/pandas/core/frame.py:781, in DataFrame.__init__(self, data, index, columns, dtype, copy)
    780 if index is None or columns is None:
--> 781     raise ValueError("DataFrame constructor not properly called!")
    783 index = ensure_index(index)

ValueErro

In [ ]:
# #datepicker callbacks
# @app.callback(Output('date-picker', 'min_date_allowed'),
#               Output('date-picker', 'max_date_allowed'),
#               Output('date-picker', 'initial_visible_month'),
#               Output('date-picker', 'date'),
#               Input('cache-data', 'data'))
# def update_datepicker(preprocessed_data):
#     if preprocessed_data is None:
#         return None, None, None, None

#     # Create a new dataframe from preprocessed_data
#     df = pd.DataFrame.from_records(preprocessed_data)

#     # Set datatype for the 'date' column
#     df['date'] = pd.to_datetime(df['date'])

#     return df['date'].min(), df['date'].max(), df['date'].max(), df['date'].max()

In [ ]:
# #displaying gauge instead of bar chart

def new_sales_chart(date, df):
    current_year = pd.Timestamp(date).year
    current_month = pd.Timestamp(date).month
    current_day = pd.Timestamp(date).day
    prev_year = current_year - 1
    two_year = current_year - 2
    
    ytd_start = pd.to_datetime(f"{current_year}-01-01")
    last_ytd_start = pd.to_datetime(f"{prev_year}-01-01")
    last_ytd_end = pd.to_datetime(f"{prev_year}-{current_month}-{current_day}")
    two_ytd_start = pd.to_datetime(f"{two_year}-01-01")
    two_ytd_end = pd.to_datetime(f"{two_year}-{current_month}-{current_day}")

    sales_ytd = round(df[(df['date'] >= ytd_start)]['total'].sum(),2)
    sales_ytd_formatted = "{:,.2f}".format(sales_ytd)
    sales_ytd_last_year = df[(df['date'] >=last_ytd_start)& (df['date'] <= last_ytd_end)]['total'].sum()
    sales_ytd_two_year = df[(df['date'] >=two_ytd_start)& (df['date'] <= two_ytd_end)]['total'].sum()
    percent_vs_last_year_to_date = round(sales_ytd/sales_ytd_last_year*100,2)
    percent_vs_two_year_to_date = sales_ytd/sales_ytd_two_year 

    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=percent_vs_last_year_to_date,
        gauge={
            'axis': {'range': [None, 100], 'tickwidth': 1, 'tickcolor': 'darkblue'},
            'bar': {'color': 'deepskyblue'},
            'bgcolor': 'white',
            'borderwidth': 2,
            'bordercolor': 'gray',
            'steps': [{'range': [0, 50], 'color': 'white'},
                      {'range': [50, 100], 'color': 'white'},
                      {'range': [100, 150], 'color': 'white'},
                      {'range': [1500, 200], 'color': 'white'}]
        },
        domain={'x': [0, 1], 'y': [0, 1]},
        #title={'text': "% of YtD vs past YtD sales"}
    ))

    # Add the sales_ytd string as a markdown string to the layout of the fig variable
    fig.update_layout(
        annotations=[
            go.layout.Annotation(
                x=0.5,
                y=1.2,
                showarrow=False,
                text=f"YTD Sales: $ {sales_ytd_formatted}",
                xref="paper",
                yref="paper",
                align='center',
                font=dict(size=18)
            ),
            go.layout.Annotation(
                x=0.5,
                y=-0.2,
                showarrow=False,
                text=f"{percent_vs_last_year_to_date}% generated vs past YtD sales",
                xref="paper",
                yref="paper",
                align='center',
                font=dict(size=18)
            )
        ]
    )
    
    return fig
new_sales_chart(today_date, df)

In [ ]:
#displaying a gauge instead of bar chart 
@app.callback(
    Output('sales-chart-2', 'figure'),
    [Input('date-picker', 'date')])
def updated_new_sales_chart(date):
    # Call the new_sales_chart function to generate the figure
    fig = new_sales_chart(date, df)
    
    return fig

In [ ]:
def create_sales_chart(df, date):
    month = pd.Timestamp(date).to_pydatetime()
    sales_mtd = df[(df['month'] == month.month) & (df['year'] == month.year)]['total'].sum()
    sales_mtd_formatted = "{:,.2f}".format(sales_mtd)
    sales_mtd_last_year = df[(df['month'] == month.month) & (df['year'] == month.year - 1)]['total'].sum()
    sales_mtd_two_year = df[(df['month'] == month.month) & (df['year'] == month.year - 2)]['total'].sum()
    percent_vs_last_year_month = round(sales_mtd/sales_mtd_last_year*100,2)
    percent_vs_two_year_month = round(sales_mtd/sales_mtd_two_year*100,2)

    labels = ['vs 2Y ago', 'vs last year','MtD']
    category_names = [sales_mtd_two_year, sales_mtd_last_year, sales_mtd_formatted]

    trace = go.Bar(
        x=category_names,
        y=labels,
        orientation='h',
        text=category_names,
        textposition='auto',
        marker=dict(
            color='deepskyblue'
        )
    )

    data = [trace]
    layout = go.Layout(
        #xaxis=dict(title='Total Sales'),
        #yaxis=dict(title='Category'),
        margin=dict(l=1)
    )

    fig = go.Figure(data=data, layout=layout)
    return fig
create_sales_chart(df, today_date)

In [ ]:
@app.callback(
    Output('sales-chart', 'figure'),
    [Input('date-picker', 'date')]
)
def update_sales_chart(date):
    month = pd.Timestamp(date).to_pydatetime()
    sales_chart = create_sales_chart(df, month)
    return sales_chart


In [ ]:
today = pd.Timestamp.now().floor('D')
def prev_weekday(adate):
    adate -= timedelta(days=1)
    while adate.weekday() > 4: # Mon-Fri are 0-4
        adate -= timedelta(days=1)
    return adate
overnight_sales = df[(df['date'] >= prev_weekday(today))]['total'].sum()
overnight_orders = df[(df['date'] >= prev_weekday(today))]['sales_order_id'].nunique()
no_not_complete = df[(df['date'] >= prev_weekday(today))& (df['state_i']!='fulfilled')]['sales_order_id'].nunique()
so_not_complete = df[(df['date'] >= prev_weekday(today))& (df['state_i']!='fulfilled')]['sales_order_id'].unique()


In [ ]:
@app.callback(
    Output('overnight-sales', 'children'),
    [Input('date-picker', 'date')]
)
def update_overnight_sales(date):
    # Calculate overnight sales based on selected date
    overnight_sales = round(df[(df['date'] >= pd.Timestamp(date).floor('D'))]['total'].sum(),2)
    overnight_sales = "{:,.2f}".format(overnight_sales)
    overnight_orders = df[(df['date'] >= pd.Timestamp(date).floor('D'))]['sales_order_id'].nunique()
    no_not_complete = df[(df['date'] >= pd.Timestamp(date).floor('D'))& (df['state_i']!='fulfilled')]['sales_order_id'].nunique()
    so_not_complete = df[(df['date'] >= pd.Timestamp(date).floor('D'))& (df['state_i']!='fulfilled')]['sales_order_id'].unique()
    return f"Overnight sales = {overnight_sales} with {overnight_orders} orders, where {no_not_complete} SO are not fulfilled : {so_not_complete}" 
update_overnight_sales(today_date)

In [ ]:
def get_customers_to_chase(df):
    # Filter for customers who last ordered this year
    current_year = datetime.now().year
    
    # How long have we got each cust
    customer_length = df.groupby('name_o').agg({'sales_order_id_o': 'nunique', 'date': ['min', 'max']})
    customer_length['longevity'] = customer_length['date']['max'] - customer_length['date']['min']
    customer_length = customer_length.reset_index()
    customer_length.columns = ['name_o', 'orders_count', 'first_order', 'last_order', 'longevity']
    customer_length['last_order'] = pd.to_datetime(customer_length['last_order'], format='%Y-%m-%d')
    filtered_df = customer_length.loc[(customer_length['last_order'].dt.year == current_year)]
    sorted_df = filtered_df[['name_o', 'orders_count', 'last_order', 'longevity']].sort_values('last_order').head(10)
    # Format the longevity column in years and months
    sorted_df['longevity'] = sorted_df['longevity'].apply(lambda x: f'{int(x.days/365)}y {int((x.days%365)/30)}m')
    sorted_df['last_order'] = pd.to_datetime(sorted_df['last_order'], format='%Y-%m-%d').dt.strftime('%d-%m-%Y')

    fig = go.Figure(data=[go.Table(
        columnwidth=1,
        header=dict(values=list(sorted_df.columns),
                    fill_color='lightgray',
                    align=['left', 'center']),
        cells=dict(values=[sorted_df.name_o, sorted_df.orders_count, sorted_df.last_order, sorted_df.longevity],
                   fill_color='white',
                   align=['left', 'center']))
    ])

    return fig

get_customers_to_chase(df)


In [ ]:
@app.callback(
    Output('table', 'figure'),
    Input('render-button', 'n_clicks'),
    #State('my-dataframe', 'data')
)
def update_table(n_clicks):
    if not n_clicks:
        raise PreventUpdate
    table = get_customers_to_chase(df)
    return table
update_table(1)

In [ ]:
df.info()

In [ ]:
df['year'].dropna().unique()

### 9- FIRST TAB - SALES DASHBOARD

In [ ]:
# Sales dashboard
sales_tab_content = dbc.Container([
     dbc.Row([
        dbc.Col(html.H6('...', className='card-title mb-4')),
        html.Label('Select year:'), 
        dcc.Dropdown(
            id='year-dropdown',
            options=[{'label': str(year), 'value': year} for year in df['year'].dropna().unique()],
            value=df['year'].min(),
            clearable=False,
            searchable=False,
            placeholder='Select a year'
        ),
        ], justify="center", align="center", className='mb-4'),

    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Sales by Top 3 Customers")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='sales-graph1')
                    ], 
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Sales vs Order vs Total SKU")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='sales-graph-2')
                    ],
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Average Order Value")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='sales-graph-3')
                    ],
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Sales by Region")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='sales-graph-4')
                    ],
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
    ]),
], fluid=True, style={'padding': '2rem'})

In [ ]:
# Calculate total sales by customer over the entire period
total_sales_by_customer = df.groupby('name_o')['total'].sum()

# Select top 20 customers based on their total sales
top_customers = total_sales_by_customer.nlargest(20).index.tolist()

# Create a dictionary of colors for the top 20 customers
customer_colors = {customer: px.colors.qualitative.Pastel1[i % len(px.colors.qualitative.Pastel1)]
                   for i, customer in enumerate(top_customers)}

# Add a color for 'others'
customer_colors['others'] = 'gray'

# Assign colors to each customer using the color mapping dictionary
def assign_color(customer):
    if customer in customer_colors:
        return customer_colors[customer]
    else:
        return 'gray'

@app.callback(
    Output('sales-graph1', 'figure'),
    [Input('year-dropdown', 'value')])
def update_sales_graph_1(selected_year):
    filtered_df = df[df['year'] == selected_year]

    # Calculate total sales by month and customer
    sales_by_month_customer = filtered_df.groupby(['month', 'name_o'])['total'].sum().reset_index()

    # Find top customers for each month
    top_customers_by_month = sales_by_month_customer.groupby('month').apply(lambda x: x.nlargest(3, 'total')).reset_index(drop=True)

    # Group the remaining customers as "others"
    remaining_customers_by_month = sales_by_month_customer[~sales_by_month_customer['name_o'].isin(top_customers)]
    remaining_customers_by_month = remaining_customers_by_month.groupby('month').sum().reset_index()
    remaining_customers_by_month['name_o'] = 'others'

    # Concatenate the top customers and the "others" dataframes
    sales_by_month_customer = pd.concat([top_customers_by_month, remaining_customers_by_month])

    # Assign colors to each customer using the color mapping dictionary
    sales_by_month_customer['color'] = sales_by_month_customer['name_o'].apply(assign_color)

    # Create stacked bar chart of sales by customer
    fig = px.bar(sales_by_month_customer, x='month', y='total', color='name_o', color_discrete_map=customer_colors)
    fig.update_layout(
        title=f'Top 3 customers and others by month ({selected_year})',
        xaxis_title='Month',
        yaxis_title='Total Sales ($)',
        barmode='stack'
    )
    return fig
update_sales_graph_1(2020)

In [ ]:
@app.callback(
    Output('sales-graph-2', 'figure'),
    [Input('year-dropdown', 'value')])
def update_sales_graph_2(selected_year):
    filtered_df = df[df['year'] == selected_year]
    
    # Total Sales trend by month
    sales_by_month = filtered_df.groupby(pd.Grouper(key='date', freq='M')).sum()['total']
    sales_trace = go.Scatter(x=sales_by_month.index, y=sales_by_month.values, mode='lines', name='Total Sales')

    # Total Unique Orders trend by month
    unique_orders_by_month = filtered_df.groupby(pd.Grouper(key='date', freq='M'))['id'].nunique()
    unique_orders_trace = go.Scatter(x=unique_orders_by_month.index, y=unique_orders_by_month.values, mode='lines', name='Total Unique Orders')

    # Total SKU trend by month
    total_sku_by_month = filtered_df.groupby(pd.Grouper(key='date', freq='M'))['quantity'].sum()
    total_sku_trace = go.Scatter(x=total_sku_by_month.index, y=total_sku_by_month.values, mode='lines', name='Total SKUs')
    
    fig = go.Figure()
    
    fig.add_trace(sales_trace)
    fig.add_trace(unique_orders_trace)
    fig.add_trace(total_sku_trace)

    fig.update_layout(
        font=dict(size=10),
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="white",
        plot_bgcolor="white",
        xaxis_title='Month',
        yaxis_title='Total',
        title="Sales, Unique Orders, and SKUs Trend"
    )

    return fig
update_sales_graph_2(2021)

### 10- SECOND TAB : CLIENT DASHBOARD

In [ ]:
clients_tab_content = dbc.Card(
    dbc.CardBody("tab to fill per client")
)

### 11- THIRD TAB : PROSPECT DASHBOARD

In [ ]:
prospects_tab_content = dbc.Card(
    dbc.CardBody("habits and nudge")
)

### 12- GLOBAL LAYOUT > END

In [ ]:
# Define tabs
tabs = dbc.Tabs(
    [
        dbc.Tab(label="Historic Sales", children=sales_tab_content),
        dbc.Tab(label="Customer Analysis", children=clients_tab_content),
        dbc.Tab(label="Prospects Market Research", children=prospects_tab_content),
    ],
    id="tabs",
    active_tab="Historic Sales",
)

In [32]:
# Define app layout
app.layout = html.Div([
    store, #dcc.Store(id='local', storage_type='local'),  # Move dcc.Store outside navbar
    navbar,
    html.Div(id='table-container'),
    page_layout,
    # dbc.Container([
    #     dbc.Row([
    #         dbc.Col([
    #             tabs,
    #         ]),
    #     ], className='mt-4')
    # ])
], style={'padding': '0rem 0rem 2rem 0rem'})

In [33]:
# # Update the page content based on the url
# @app.callback(Output("page-content", "children"), [Input("url", "pathname")])
# def render_page_content(pathname):
#     if pathname == "/":
#         return landing_page
#     elif pathname == "/apps/sales":
#         return sales_dashboard
#     elif pathname == "/apps/clients":
#         return clients_dashboard
#     elif pathname == "/apps/prospects":
#         return prospects_dashboard
#     else:
#         return html.P("Page not found")


In [34]:
# Set the app layout
#app.layout = app_layout
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False, port=8077)

Dash is running on http://127.0.0.1:8077/

 * Serving Flask app '__main__'
 * Debug mode: on
